# Algorithm 1

First allocation algorithm implemented 🚀 (Only problem: I have no clue how it works 😂) 
- only events with one hall
- only one parking lot per event (actually not 100% sure)
- only one parking lot per event (thats why fre.e + IMOT doesn't have a parking lot)
- algorithm minimizes the distance in the entire dataframe not per day (must be fixed for sure)

## Import Libraries

In [1]:
# Import Libraries
import pulp as pl
import pandas as pd
import pulp as pl

# Define Algorithm

In [2]:
def optimize_distance(df_events_closest_parking_lot_min_capacity):

    # Define the problem
    model = pl.LpProblem("Minimize_Distance", pl.LpMinimize)

    # Define decision variables
    assignments = pl.LpVariable.dicts(
        "Assign",
        (
            (event, date, parking_lot)
            for index, (
                event,
                date,
                parking_lot,
            ) in df_events_closest_parking_lot_min_capacity[
                ["event", "date", "parking_lot"]
            ]
            .drop_duplicates()
            .iterrows()
        ),
        cat="Binary",
    )

    # Objective: Minimize the total minimum distance
    model += pl.lpSum(
        assignments[(event, date, parking_lot)] * min_distance
        for index, (
            event,
            date,
            parking_lot,
            min_distance,
        ) in df_events_closest_parking_lot_min_capacity[
            ["event", "date", "parking_lot", "min_distance"]
        ].iterrows()
    )

    # Constraint: Each event on each date should have exactly one parking lot assigned
    for (event, date), group in df_events_closest_parking_lot_min_capacity.groupby(
        ["event", "date"]
    ):
        model += (
            pl.lpSum(
                assignments[(event, date, parking_lot)]
                for parking_lot in group["parking_lot"]
            )
            == 1
        )

    # Constraint: Do not exceed the capacity of any parking lot on any given day
    for (
        parking_lot,
        date,
    ), group in df_events_closest_parking_lot_min_capacity.groupby(
        ["parking_lot", "date"]
    ):
        model += (
            pl.lpSum(
                assignments[(event, date, parking_lot)] * demand
                for index, (event, demand) in group[["event", "demand"]].iterrows()
            )
            <= group["capacity"].iloc[0]
        )

    # Solve the model
    model.solve()

    # Output results
    df_allocation_results = []
    for (event, date, parking_lot), var in assignments.items():
        if pl.value(var) == 1:
            df_allocation_results.append(
                {"event": event, "date": date, "parking_lot": parking_lot}
            )

    df_allocation_results = pd.DataFrame(df_allocation_results)

    # merge capacity onto df_allocation_results
    df_allocation_results = df_allocation_results.merge(
        df_events_closest_parking_lot_min_capacity[["parking_lot"]].drop_duplicates(),
        on="parking_lot",
    )

    return df_allocation_results

## Load the data needed

In [3]:
# Load the data
df_events_closest_parking_lot_min_capacity = pd.read_parquet('../data/output_data/df_events_closest_parking_lot_min_capacity.parquet')
df_events_closest_parking_lot_min_capacity.head()

,event,date,status,demand,hall,parking_lot,distance_north,distance_north_east,distance_east,distance_west,distance_north_west,capacity,parking_delta,min_distance,closest_entrance,max_demand
0,inhorgenta,2025-02-17,aufbau,280,A1,P1 Nord (Tor 17a - Tor 11c),600,730,760,1060,1000,2750,2470,600,north,2576
1,inhorgenta,2025-02-17,aufbau,280,A1,P1 Nord (westl. Tor 17a),580,710,740,1010,950,350,70,580,north,2576
2,inhorgenta,2025-02-17,aufbau,280,A1,P2 Nord (östl. Tor 11c),620,690,660,1360,1300,500,220,620,north,2576
3,inhorgenta,2025-02-17,aufbau,280,A1,P7,850,970,850,1260,1200,400,120,850,north,2576
4,inhorgenta,2025-02-17,aufbau,280,A1,P9 - P12,700,830,720,1160,1150,3000,2720,700,north,2576


## Run the optimization

In [4]:
df_allocation_results = optimize_distance(df_events_closest_parking_lot_min_capacity)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jakobschwarz/Documents/01_study/02_master/01_coursework/02_semester/02_Management and Digital Technologies II - Software Development Project (12)/development/southpark/venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0x/vxtbhltd3k1g7dgrlry82kfh0000gn/T/64ebae93e2b04ad2be99b5079653a38b-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/0x/vxtbhltd3k1g7dgrlry82kfh0000gn/T/64ebae93e2b04ad2be99b5079653a38b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 134 COLUMNS
At line 1050 RHS
At line 1180 BOUNDS
At line 1366 ENDATA
Problem MODEL has 129 rows, 185 columns and 360 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 18630.7 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 20 strengthened rows, 1 substitutions
Cgl0003I 0 fixed, 0 

In [5]:
df_allocation_results.head()

,event,date,parking_lot
0,inhorgenta,2025-02-17,P1 Nord (westl. Tor 17a)
1,inhorgenta,2025-02-18,P1 Nord (westl. Tor 17a)
2,inhorgenta,2025-02-19,P1 Nord (Tor 17a - Tor 11c)
3,inhorgenta,2025-02-20,P1 Nord (Tor 17a - Tor 11c)
4,inhorgenta,2025-02-21,P1 Nord (Tor 17a - Tor 11c)


## Merge Allocation results, Event data, Parking Lot distances and Parking lot capacity

In [6]:
# Load events data for merging with the results
df_events = pd.read_csv('../data/input_data/events.csv')
df_events['date'] = pd.to_datetime(df_events['date'], format='%d.%m.%y')

In [7]:
# load df_halls_parking_distances
df_halls_parking_distances = pd.read_parquet('../data/output_data/halls_parking_distances.parquet')

# load parking lots capacity
df_parking_lots_capacity = pd.read_csv('../data/input_data/parking_lots_capacity.csv')

# Merge the results with the events data
df_events_allocated_parking_lot = pd.merge(df_events, df_allocation_results, on=['event', 'date'], how='left')

# Merge df_events_allocated_parking_lot with df_events_closest_parking_lot_min_capacity to get distances
df_events_allocated_parking_lot = pd.merge(df_events_allocated_parking_lot, df_halls_parking_distances, on=['hall', 'parking_lot'], how='left')

# Merge capacity from df_events_closest_parking_lot_min_capacity onto df_events_allocated_parking_lot

df_events_allocated_parking_lot = pd.merge(
    df_events_allocated_parking_lot, 
    df_events_closest_parking_lot_min_capacity[['parking_lot', 'date', 'capacity']], 
    on=['parking_lot', 'date'], 
    how='left'
).drop_duplicates()

In [8]:
# Add parking_delta column
df_events_allocated_parking_lot['parking_delta'] = df_events_allocated_parking_lot['capacity'] - df_events_allocated_parking_lot['demand']

# Add a column for the minimum distance and the entrance
df_events_allocated_parking_lot['min_distance'] = df_events_allocated_parking_lot[['distance_north', 'distance_north_east', 'distance_east', 'distance_west', 'distance_north_west']].min(axis=1)

# Check if there are any rows with all-NA values
mask = df_events_allocated_parking_lot[['distance_north', 'distance_north_east', 'distance_east', 'distance_west', 'distance_north_west']].notna().any(axis=1)

# Only calculate 'closest_entrance' for rows without all-NA values
df_events_allocated_parking_lot.loc[mask, 'closest_entrance'] = df_events_allocated_parking_lot.loc[mask, ['distance_north', 'distance_north_east', 'distance_east', 'distance_west', 'distance_north_west']].idxmin(axis=1).str.replace('distance_', '')

In [9]:
# First allocation algorithm results
df_events_allocated_parking_lot[df_events_allocated_parking_lot['date'] == pd.to_datetime('2025-02-23')][['event', 'date', 'status', 'hall', 'demand', 'parking_delta', 'capacity', 'parking_lot', 'min_distance', 'closest_entrance']].head(20)

,event,date,status,hall,demand,parking_delta,capacity,parking_lot,min_distance,closest_entrance
12,inhorgenta,2025-02-23,laufzeit,A1,1795,955.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north
41,Münchner Autotage,2025-02-23,laufzeit,B3,1000,2000.0,3000.0,P9 - P12,600.0,north
67,fre.e + IMOT,2025-02-23,laufzeit,B4,12724,NaN,NaN,NaN,NaN,NaN
75,Lopec,2025-02-23,aufbau,C6,140,360.0,500.0,P2 Nord (östl. Tor 11c),440.0,north_east


In [10]:
# Verify, that there are no negative parking delta
if (df_events_allocated_parking_lot['parking_delta'] < 0).any():
    print(df_events_allocated_parking_lot[df_events_allocated_parking_lot['parking_delta'] < 0])
else:
    print('No event demand exceeds allocated parking capacity')

No event demand exceeds allocated parking capacity


In [11]:
df_events_allocated_parking_lot[['event', 'date', 'status', 'hall', 'demand', 'parking_delta', 'capacity', 'parking_lot', 'min_distance', 'closest_entrance']].head(100)

,event,date,status,hall,demand,parking_delta,capacity,parking_lot,min_distance,closest_entrance
0,inhorgenta,2025-02-17,aufbau,A1,280,70.0,350.0,P1 Nord (westl. Tor 17a),580.0,north
2,inhorgenta,2025-02-18,aufbau,A1,330,20.0,350.0,P1 Nord (westl. Tor 17a),580.0,north
4,inhorgenta,2025-02-19,aufbau,A1,420,2330.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north
6,inhorgenta,2025-02-20,aufbau,A1,420,2330.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north
8,inhorgenta,2025-02-21,laufzeit,A1,2278,472.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north
10,inhorgenta,2025-02-22,laufzeit,A1,2576,174.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north
12,inhorgenta,2025-02-23,laufzeit,A1,1795,955.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north
15,inhorgenta,2025-02-24,laufzeit,A1,1828,922.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north
19,inhorgenta,2025-02-25,abbau,A1,340,2410.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north
23,inhorgenta,2025-02-26,abbau,A1,332,18.0,350.0,P1 Nord (westl. Tor 17a),580.0,north


In [12]:
# Group by 'date' and 'parking_lot' and count occurrences
parking_lot_occurrences = df_events_allocated_parking_lot.groupby(['date', 'parking_lot']).size()

# Filter to find any date-parking lot combinations with more than one occurrence
duplicate_parking_lots = parking_lot_occurrences[parking_lot_occurrences > 1]

# Display the results
print(duplicate_parking_lots)


date        parking_lot                
2025-02-17  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-18  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-19  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-20  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-24  P1 Nord (westl. Tor 17a)       2
2025-02-25  P1 Nord (westl. Tor 17a)       2
dtype: int64


# Algorithm 2 (WIP)

This currently assigns only one parking lot per event but it also assigns the same parking lot to multiple events

In [13]:
def optimize_distance(df_events_closest_parking_lot_min_capacity):
    # Filter out rows where parking lot capacity is less than the max demand for the event
    feasible_allocations = df_events_closest_parking_lot_min_capacity[
        df_events_closest_parking_lot_min_capacity['capacity'] >= df_events_closest_parking_lot_min_capacity['max_demand']
    ]

    # Define the problem
    model = pl.LpProblem("Minimize_Distance", pl.LpMinimize)

    # Define decision variables for feasible allocations only
    assignments = pl.LpVariable.dicts(
        "Assign",
        (
            (event, parking_lot)
            for event, parking_lot in feasible_allocations[['event', 'parking_lot']].drop_duplicates().itertuples(index=False)
        ),
        cat="Binary"
    )

    # Objective: Minimize the total minimum distance
    model += pl.lpSum(
        assignments[(event, parking_lot)] * pl.lpSum(
            min_distance for _, (ev, dt, pl, min_distance) in 
            feasible_allocations[feasible_allocations['event'] == event][['event', 'date', 'parking_lot', 'min_distance']].iterrows() if pl == parking_lot
        )
        for event, parking_lot in assignments
    )

    # Constraint: Each event should have exactly one parking lot assigned
    for event in feasible_allocations['event'].unique():
        model += pl.lpSum(
            assignments[(event, parking_lot)] for parking_lot in 
            feasible_allocations[feasible_allocations['event'] == event]['parking_lot'].unique()
        ) == 1

    # Solve the model
    model.solve()

    # Output results
    df_allocation_results = []
    for (event, parking_lot), var in assignments.items():
        if pl.value(var) == 1:
            event_dates = feasible_allocations[
                (feasible_allocations['event'] == event) & 
                (feasible_allocations['parking_lot'] == parking_lot)
            ]['date'].unique()
            for date in event_dates:
                df_allocation_results.append({
                    "event": event,
                    "date": date,
                    "parking_lot": parking_lot
                })


    df_allocation_results = pd.DataFrame(df_allocation_results)

    return df_allocation_results


## Load the data needed

In [14]:
# Load the data again
df_events_closest_parking_lot_min_capacity = pd.read_parquet('../data/output_data/df_events_closest_parking_lot_min_capacity.parquet')

In [15]:
df_events_closest_parking_lot_min_capacity.head()

,event,date,status,demand,hall,parking_lot,distance_north,distance_north_east,distance_east,distance_west,distance_north_west,capacity,parking_delta,min_distance,closest_entrance,max_demand
0,inhorgenta,2025-02-17,aufbau,280,A1,P1 Nord (Tor 17a - Tor 11c),600,730,760,1060,1000,2750,2470,600,north,2576
1,inhorgenta,2025-02-17,aufbau,280,A1,P1 Nord (westl. Tor 17a),580,710,740,1010,950,350,70,580,north,2576
2,inhorgenta,2025-02-17,aufbau,280,A1,P2 Nord (östl. Tor 11c),620,690,660,1360,1300,500,220,620,north,2576
3,inhorgenta,2025-02-17,aufbau,280,A1,P7,850,970,850,1260,1200,400,120,850,north,2576
4,inhorgenta,2025-02-17,aufbau,280,A1,P9 - P12,700,830,720,1160,1150,3000,2720,700,north,2576


## Run the optimization

In [16]:
# Run the new allocation algorithm
df_allocation_results_2 = optimize_distance(df_events_closest_parking_lot_min_capacity)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jakobschwarz/Documents/01_study/02_master/01_coursework/02_semester/02_Management and Digital Technologies II - Software Development Project (12)/development/southpark/venv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/0x/vxtbhltd3k1g7dgrlry82kfh0000gn/T/18be89406d2b44dabc9b7e0707055bdd-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/0x/vxtbhltd3k1g7dgrlry82kfh0000gn/T/18be89406d2b44dabc9b7e0707055bdd-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 54 RHS
At line 59 BOUNDS
At line 71 ENDATA
Problem MODEL has 4 rows, 11 columns and 11 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 23040 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No inte

## Merge Allocation results, Event data, Parking Lot distances and Parking lot capacity

In [17]:
# load df_halls_parking_distances
df_halls_parking_distances = pd.read_parquet('../data/output_data/halls_parking_distances.parquet')

# load parking lots capacity
df_parking_lots_capacity = pd.read_csv('../data/input_data/parking_lots_capacity.csv')

# Merge the results with the events data
df_events_allocated_parking_lot = pd.merge(df_events, df_allocation_results_2, on=['event', 'date'], how='left')

# Merge df_events_allocated_parking_lot with df_events_closest_parking_lot_min_capacity to get distances
df_events_allocated_parking_lot = pd.merge(df_events_allocated_parking_lot, df_halls_parking_distances, on=['hall', 'parking_lot'], how='left')

# Merge capacity from df_events_closest_parking_lot_min_capacity onto df_events_allocated_parking_lot
df_events_allocated_parking_lot = pd.merge(
    df_events_allocated_parking_lot, 
    df_events_closest_parking_lot_min_capacity[['parking_lot', 'date', 'capacity']], 
    on=['parking_lot', 'date'], 
    how='left'
).drop_duplicates()

In [18]:
# Add parking_delta column
df_events_allocated_parking_lot['parking_delta'] = df_events_allocated_parking_lot['capacity'] - df_events_allocated_parking_lot['demand']

# Add a column for the minimum distance and the entrance
df_events_allocated_parking_lot['min_distance'] = df_events_allocated_parking_lot[['distance_north', 'distance_north_east', 'distance_east', 'distance_west', 'distance_north_west']].min(axis=1)

# Check if there are any rows with all-NA values
mask = df_events_allocated_parking_lot[['distance_north', 'distance_north_east', 'distance_east', 'distance_west', 'distance_north_west']].notna().any(axis=1)

# Only calculate 'closest_entrance' for rows without all-NA values
df_events_allocated_parking_lot.loc[mask, 'closest_entrance'] = df_events_allocated_parking_lot.loc[mask, ['distance_north', 'distance_north_east', 'distance_east', 'distance_west', 'distance_north_west']].idxmin(axis=1).str.replace('distance_', '')

# Add max_demand to df_events_allocated_parking_lot
df_events_allocated_parking_lot['max_demand'] = df_events_allocated_parking_lot.groupby('event')['demand'].transform('max')


## Output

In [19]:
# Alternative allocation algorithm results does not solve the problem
pd.set_option('display.max_rows', None)
df_events_allocated_parking_lot[['event', 'date', 'status', 'hall', 'demand', 'parking_delta', 'capacity', 'parking_lot', 'min_distance', 'closest_entrance', 'max_demand']].head(100).reset_index(drop=True)

,event,date,status,hall,demand,parking_delta,capacity,parking_lot,min_distance,closest_entrance,max_demand
0,inhorgenta,2025-02-17,aufbau,A1,280,2470.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north,2576
1,inhorgenta,2025-02-18,aufbau,A1,330,2420.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north,2576
2,inhorgenta,2025-02-19,aufbau,A1,420,2330.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north,2576
3,inhorgenta,2025-02-20,aufbau,A1,420,2330.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north,2576
4,inhorgenta,2025-02-21,laufzeit,A1,2278,472.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north,2576
5,inhorgenta,2025-02-22,laufzeit,A1,2576,174.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north,2576
6,inhorgenta,2025-02-23,laufzeit,A1,1795,955.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north,2576
7,inhorgenta,2025-02-24,laufzeit,A1,1828,922.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north,2576
8,inhorgenta,2025-02-25,abbau,A1,340,2410.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north,2576
9,inhorgenta,2025-02-26,abbau,A1,332,2418.0,2750.0,P1 Nord (Tor 17a - Tor 11c),600.0,north,2576


In [20]:
# Group by 'date' and 'parking_lot' and count occurrences
parking_lot_occurrences = df_events_allocated_parking_lot.groupby(['date', 'parking_lot']).size()

# Filter to find any date-parking lot combinations with more than one occurrence
duplicate_parking_lots = parking_lot_occurrences[parking_lot_occurrences > 1]

# Display the results
print(duplicate_parking_lots)


date        parking_lot                
2025-02-17  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-18  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-19  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-20  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-21  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-22  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-23  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-24  P1 Nord (Tor 17a - Tor 11c)    2
2025-02-25  P1 Nord (Tor 17a - Tor 11c)    2
dtype: int64
